# A mobile app for lottery addiction

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the 6/49 lottery and build functions that enable users to answer questions like:

- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

### Core Functions

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. The drawing is done without replacement, which means once a number is drawn, it's not put back in the set.

We'll use two principal function for calculate `factorial` and `combinations`.

In [23]:
def factorial(n):
    """
    Calculate the factorial of n (n!)
    Args:
        n: numeric value
    Returns:
        The n!.
    Examples
    ________
    >>> factorial(5)
    120
    """
    if n == 1:
        return 1
    return n * factorial(n - 1)

In [2]:
def combinations(n, k):
    """
    Calculate the combinations of n taking k objects.
    Args:
        n: number represent a group.
        k: number of takin objectcs from group.
    Returns:
        The nCk.
    Examples
    ________
    >>> combinations(52, 5)
    2598960.0
    """
    return factorial(n) / (factorial(k) * factorial(n - k))


##  One-ticket Probability
We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

- Inside the app, the user inputs six different numbers from 1 to 49.
- Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
- The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.

In [3]:
def one_ticket_probability(numbers):
    outcomes = combinations(49, 6)
    successful_outcome = 1 / outcomes
    percentage_win = successful_outcome * 100
    print(f'The probability of the numbers {numbers} winning the grand prize is {percentage_win:.6f}%.')
    print(f'You have 1 in  {outcomes:,} chances to win.')

In [4]:
one_ticket_probability(['13', '17', '19', '10', '30', '43'])

The probability of the numbers ['13', '17', '19', '10', '30', '43'] winning the grand prize is 0.000007%.
You have 1 in  13,983,816.0 chances to win.


In [5]:
one_ticket_probability(['3', '1', '4', '1', '6', '5'])

The probability of the numbers ['3', '1', '4', '1', '6', '5'] winning the grand prize is 0.000007%.
You have 1 in  13,983,816.0 chances to win.


## Historical Data Check for Canada Lottery
Users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

We'll focus on exploring the historical data coming from the Canada 6/49 lottery. The data set can be downloaded from [Kaggle](https://www.kaggle.com/datascienceai/lottery-dataset) and it has the following structure:

In [6]:
import pandas as pd
lottery = pd.read_csv('649.csv')

In [7]:
lottery.shape

(3665, 11)

In [8]:
lottery.head()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34
3,649,4,0,7/3/1982,3,9,10,13,20,43,34
4,649,5,0,7/10/1982,5,14,21,31,34,47,45


In [9]:
lottery.tail()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3660,649,3587,0,6/6/2018,10,15,23,38,40,41,35
3661,649,3588,0,6/9/2018,19,25,31,36,46,47,26
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


##  Function for Historical Data Check

We're going to write a function that will enable users to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

We'll define a function that takes as input a row of the lottery dataframe and returns a set containing all the six winning numbers.

In [10]:
def extract_numbers(row: pd.Series):
    return {i for i in row[4:10]}

In [11]:
winning_numbers = lottery.apply(extract_numbers, axis=1)
winning_numbers

0        {3, 41, 11, 12, 43, 14}
1        {33, 36, 37, 39, 8, 41}
2         {1, 6, 39, 23, 24, 27}
3         {3, 9, 10, 43, 13, 20}
4        {34, 5, 14, 47, 21, 31}
                  ...           
3660    {38, 40, 41, 10, 15, 23}
3661    {36, 46, 47, 19, 25, 31}
3662     {32, 34, 6, 22, 24, 31}
3663     {2, 38, 15, 49, 21, 31}
3664    {35, 37, 14, 48, 24, 31}
Length: 3665, dtype: object

In [12]:
def check_historical_occurence(user_numbers:[int], winning_combinations:pd.Series):
    numbers = set(user_numbers)
    winning_matches = winning_combinations == numbers
    count_matches = winning_matches.sum()
    if count_matches:
        print(f'In the past the combination {user_numbers:} occurred in the past {count_matches} time(s).')
    else:
        print(f'In the past the combination {user_numbers} never occurred in the past.')
    
    print('With the combination you selected, you have 1 in 13,983,816 possibilities to win the big price.')

In [13]:
check_historical_occurence([3, 11, 12, 14, 41, 43], winning_numbers)

In the past the combination [3, 11, 12, 14, 41, 43] occurred in the past 1 time(s).
With the combination you selected, you have 1 in 13,983,816 possibilities to win the big price.


In [14]:
check_historical_occurence([13, 17, 19, 30, 10, 30], winning_numbers)

In the past the combination [13, 17, 19, 30, 10, 30] never occurred in the past.
With the combination you selected, you have 1 in 13,983,816 possibilities to win the big price.


##  Multi-ticket Probability

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning. We're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

In [17]:
def multi_ticket_probability(number_tickets:int):
    outcomes = combinations(49, 6) 
    probability = number_tickets / outcomes
    probability_presentable = probability * 100
    
    print(f'With {number_tickets:,} ticket(s) you have a probability of winning of {probability_presentable:.6f}%.') 
    print(f'You have {number_tickets:,} in  {outcomes:,} chances to win.')

In [18]:
test = [1, 10, 100, 10000, 1000000, 6991908, 13983816]

for t in test:
    multi_ticket_probability(t)
    print('\n---------------------------------------')

With 1 ticket(s) you have a probability of winning of 0.000007%.
You have 1 in  13,983,816.0 chances to win.

---------------------------------------
With 10 ticket(s) you have a probability of winning of 0.000072%.
You have 10 in  13,983,816.0 chances to win.

---------------------------------------
With 100 ticket(s) you have a probability of winning of 0.000715%.
You have 100 in  13,983,816.0 chances to win.

---------------------------------------
With 10,000 ticket(s) you have a probability of winning of 0.071511%.
You have 10,000 in  13,983,816.0 chances to win.

---------------------------------------
With 1,000,000 ticket(s) you have a probability of winning of 7.151124%.
You have 1,000,000 in  13,983,816.0 chances to win.

---------------------------------------
With 6,991,908 ticket(s) you have a probability of winning of 50.000000%.
You have 6,991,908 in  13,983,816.0 chances to win.

---------------------------------------
With 13,983,816 ticket(s) you have a probability of

## Less Winning Numbers — Function

For extra context, in most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

These are the engineering details we'll need to be aware of:

- Inside the app, the user inputs:
  - six different numbers from 1 to 49; and
  - an integer between 2 and 5 that represents the number of winning numbers expected
- Our function prints information about the probability of having the inputted number of winning numbers.

In [53]:
def probability_less_6(winning_numbers:int):
    outcomes = combinations(49, 6)
    
    number_combinations = combinations(6, winning_numbers)
    
    non_winning_numbers = 49 - 6
    remaining_object_take = 6 - winning_numbers
    combinations_remaining = combinations(non_winning_numbers,remaining_object_take)
    print()
    successful_outcomes = number_combinations * combinations_remaining

    probability = successful_outcomes / outcomes
    probability_presentable = probability * 100
    
    scale_combinations = round(outcomes/successful_outcomes)    

    
    print(f'The probabilities that you have {winning_numbers} winning numbers are {probability_presentable:.6f}%.')
    print(f'You have 1 in  {scale_combinations:,} chances to win.')

In [54]:
test = [2, 3, 4, 5]

for t in test:
    probability_less_6(t)
    print('\n---------------------------------------')


The probabilities that you have 2 winning numbers are 13.237803%.
You have 1 in  1,851,150.0 chances to win.

---------------------------------------

The probabilities that you have 3 winning numbers are 1.765040%.
You have 1 in  246,820.0 chances to win.

---------------------------------------

The probabilities that you have 4 winning numbers are 0.096862%.
You have 1 in  13,545.0 chances to win.

---------------------------------------

The probabilities that you have 5 winning numbers are 0.001845%.
You have 1 in  258.0 chances to win.

---------------------------------------
